In [1]:
import numpy as np
from numpy import random
from scipy.stats import norm
import torch
import torch.nn as nn
#import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import TensorDataset, DataLoader, random_split
import math
from argparse import Namespace
import sys
from tqdm.notebook import tqdm
import gc

import pandas as pd
import traceback
import os  
from joblib import Parallel, delayed
#sys.path.append("/home/yuedu/deep-my")
# sys.path.append("/work/home/ac7hjj2nfu/project/indtest")
#sys.path.append("C:\\Users\\duyue\\.spyder-py3")


#from fun_all import generate_one_minus_one_distribution

import itertools
import time 
import fun_all
import gene_data_Copy1
import non_connect_nn_l1
import multiprocessing as mp
import matplotlib.pyplot as plt

In [2]:
import traceback
import smtplib
import email.utils
from email.mime.text import MIMEText
def send_email(subject, content):
    message = MIMEText(content)
    message['To'] = email.utils.formataddr(('Duuuuu', '3013081232@qq.com'))
    message['From'] = email.utils.formataddr(('Duuuuu', '3013081232@qq.com'))
    message['Subject'] = subject
    server = smtplib.SMTP_SSL('smtp.qq.com', 465)
    server.login('3013081232@qq.com', 'ysvbwhdzhtkzdfcc')
    try:
        server.sendmail('3013081232@qq.com', ['3013081232@qq.com'], msg=message.as_string())
        server.quit()
        print("邮件发送成功")
    except Exception as e:
        print("邮件发送失败:", e)

In [3]:
import logging

def init_logging(log_file):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    # 文件处理器
    file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
    file_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    file_handler.setFormatter(formatter)
    
    # 控制台处理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    
    # 添加处理器
    if not logger.handlers:
        logger.addHandler(file_handler)
        logger.addHandler(console_handler)
    
    return logger

In [4]:
def threshold(mat, thresh):
    mat = torch.clamp(mat, min=-thresh, max=thresh)
    return mat
def trainer(i,kk,exp_v, res_v, n_1, n_2, batchsize, val_pct, lr, n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device, thresh=None, plot=False):
    logger = init_logging("logex6_nn.log")  # 为每个任务初始化日志
    criterion = nn.MSELoss()
    model = non_connect_nn_l1.AdaptiveNet(input_features, hidden_features1, hidden_features2, output_features).to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer = torch.optim.Adam(model.parameters(),
                                   lr=lr,
                                   weight_decay=0.)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
    #                                           step_size=20,
    #                                           gamma=0.9)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, min_lr=1e-4)
    # 定义优化器，这里使用Adam优化器

    #dataset = TensorDataset(exp_v[n_1:(n_1+n_2),:], res_v[n_1:(n_1+n_2)])
    train_dataset = TensorDataset(exp_v[n_1:(n_1+n_2),:], res_v[n_1:(n_1+n_2)])
    
    n1_ind = torch.arange(0, n_1)
    n2_ind = torch.arange(n_1+n_2, n)
    all_indices = torch.cat([n1_ind, n2_ind])
    val_dataset = TensorDataset(exp_v[all_indices,:], res_v[all_indices])
    # val_size = int(val_pct * len(dataset))
    # train_size = len(dataset) - val_size
    # train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True,drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
    run_loss = 0 
    
    min_val_loss = float('inf')  # 保存最小验证损失
    no_improvement_epochs = 0  # 记录验证损失未改善的次数
    
    val_losses = [] 
    run_losses = []
    for epoch in range(n_epochs):
        model.train()
        run_loss1=run_loss
        run_loss = 0
    
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels)
            # print(outputs.squeeze().shape, labels.shape)
            loss.backward()
            optimizer.step()
            run_loss += loss.item()
    
        run_loss =  run_loss / len(train_loader)
        run_losses.append(run_loss)
        #print(f"the {epoch+1}, TRAIN loss {run_loss}")
    
         #验证
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)
    
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs.view(-1), val_labels).item()
        val_loss = val_loss / len(val_loader)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            no_improvement_epochs = 0  # 重置
            best_model_state = {k: v.clone() for k, v in model.state_dict().items()}
            # torch.save(model_all.state_dict(), 'best_model.pth')  # 保存最佳模型
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience or min_val_loss<1e-3:
            #print(f"Early stopping at epoch {epoch + 1}, validation loss did not improve for {patience} epochs. {min_val_loss}")
            break
    
    model.load_state_dict(best_model_state)
    if thresh:
        with torch.no_grad():
            model.eval()
            fit = threshold(model.forward(exp_v.to(device)),thresh)
    else:
        with torch.no_grad():
            model.eval()
            fit = model(exp_v.to(device))
    error = res_v.to(device) - fit.squeeze()
    if i % 100 == 0 and kk%100==0:
        logger.info(f"第{kk}次的{i}网络完成")  # 写入日志
    if plot:
        fig = plt.figure(figsize=(10, 4))
        plt.style.use('fivethirtyeight')
        plt.plot(run_losses,
                 label='Training Loss', color='C0', linewidth=2)
        plt.plot(val_losses,
                 label='Validation Loss', color='C1', linewidth=2)
        plt.yscale('log')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.show()
    return {'model': model,
                'error': error,
                'losses': run_losses,
                'val_losses': val_losses,}

In [5]:
def gauss_fun(kk,x, y, z, n, n_1, n_2, n_3, n_tilde, p, q, m, dep, alpha,batchsize,val_pct,lr,n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device):
    x = fun_all.data_transform(x[0:n_1,:], x)
    y = fun_all.data_transform(y[0:n_1,:], y)
    z = fun_all.data_transform(z[0:n_1,:], z)
    sA = torch.zeros(3, 3)
    rep_xy = torch.cat([x, y], dim=1)
    thresh = (math.log(m*n))**(1/2) * math.log(n)
    results = [trainer(i,kk,z, rep_xy[:, i], n_1, n_2, batchsize, val_pct, lr, n_epochs, patience, 
                       input_features, hidden_features1, hidden_features2, output_features, device,thresh) for i in range(p + q)]
    
    error_all = torch.stack([res['error'] for res in results],dim=1)
    eps_all = error_all[:, 0:p]
    delta_all = error_all[:, p:(p+q)]
    
    ell = n_tilde.shape[0]
    N=5000
    N_1 = 300
    res1_red = torch.zeros(N_1, ell)
    res1_norm = torch.zeros(N_1, ell)
    res1_mm = torch.zeros(N_1, ell)
    ########data driven
    for t in range(N_1):
        # torch.manual_seed((t+1)*54321)
        f1 = (torch.randn(n, n) / (n**(1/2))).to(device)
        f2 = (torch.randn(n, n) / (n**(1/2))).to(device)
        f3 = (torch.randn(n, n) / (n**(1/2))).to(device)
        w_hs_norm = f1 @ z.to(device)
        eps_norm = f2 @ eps_all
        delta_norm = f3 @ delta_all
        with torch.no_grad():
            f_h_hs = torch.stack([res['model'].forward(w_hs_norm.to(device)).squeeze() for res in results],dim=1)
        thresh1 = norm.ppf((1-1/n_1), 0, 1)
        w_hs_norm  = threshold(w_hs_norm, thresh1)
        u_hs_norm = f_h_hs[:, 0: p] + eps_norm
        u_hs_norm = threshold(u_hs_norm, thresh1)
        v_hs_norm = f_h_hs[:, p: (p+q)] + delta_norm
        v_hs_norm = threshold(v_hs_norm, thresh1)
        if t==0:
            rep_uv = torch.cat([u_hs_norm, v_hs_norm], dim=1)
            results2 = [trainer(k,t,w_hs_norm, rep_uv[:, k], n_1, n_2, batchsize, val_pct, lr, n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device,thresh) for k in range(p + q)]
        ###内部训练神经网络
        with torch.no_grad():
            f_hhm = torch.stack([res['model'].forward(w_hs_norm.to(device)).squeeze() for res in results2],dim=1)
        error_all_u = torch.cat([u_hs_norm, v_hs_norm], dim=1) - f_hhm
        eps_hhs = error_all_u[(n_1+n_2):n, 0:p]
        delta_hhs = error_all_u[(n_1+n_2):n, p:(p+q)]    
        M = fun_all.generate_one_minus_one_distribution(N, n_3).float().to(device)
        M_norm = torch.randn(N, n_3).to(device) 
        M_mm = fun_all.generate_mammen(N,n_3).float().to(device)
        for i in range(ell):
            ###critical value
            n_tilde1 = n_tilde[i]
            sqrt_n_tilde1 = torch.sqrt(torch.as_tensor(n_tilde1)).to(device)
            dat_new = torch.empty(N, p).to(device)
            dat_new_n = torch.empty(N, p).to(device)
            dat_new_mm = torch.empty(N, p).to(device)
            for j in range(p):
                Ai = eps_hhs[0:n_tilde1, j:j+1]
                dat_i = Ai * delta_hhs[0:n_tilde1, :]  # Hadamard 内积，形状 (n_tilde1, q)
                dat_i = dat_i - torch.mean(dat_i, dim=0, keepdim=True)  # 减去均值
                result_i = torch.max(abs(M[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new[:, j] = result_i  # 存储结果
                result_i_n = torch.max(abs(M_norm[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new_n[:, j] = result_i_n  # 存储结果
                result_i_mm = torch.max(abs(M_mm[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new_mm[:, j] = result_i_mm  # 存储结果
            dat_new = torch.max(dat_new, axis = 1).values
            dat_new_n = torch.max(dat_new_n, axis = 1).values
            dat_new_mm = torch.max(dat_new_mm, axis = 1).values
            # dat = torch.einsum('np,nq->npq', eps_hhs[0:n_tilde1, :], delta_hhs[0:n_tilde1, :]).reshape(n_tilde1, -1)  # (n_tilde1, p * q)
            # # dat = eps_hhs[0:n_tilde1, :].repeat_interleave(q, dim=1) * delta_hhs[0:n_tilde1, :].repeat(1, p)
            # dat = dat - torch.mean(dat, dim=0)
            # #rademacher 
            # dat_new = torch.max(abs(M[:,0:n_tilde1].float() @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_red = torch.kthvalue(dat_new, (N-int(N*alpha)+1)).values
                #norm
            # dat_new_n = torch.max(abs(M_norm[:,0:n_tilde1] @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_norm = torch.kthvalue(dat_new_n, (N-int(N*alpha)+1)).values
            #norm
            # dat_new_mm = torch.max(abs(M_mm[:,0:n_tilde1].float() @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_mm = torch.kthvalue(dat_new_mm, (N-int(N*alpha)+1)).values
            #test-statistics
            test1 = np.sqrt(n_tilde1) * torch.max(abs(eps_hhs[0:n_tilde1, :].T @ delta_hhs[0:n_tilde1, :] / n_tilde1))
            res1_red[t, i]  = test1 >cv_red
            res1_norm[t, i] = test1 >cv_norm
            res1_mm[t, i]   = test1 >cv_mm
            #if t == 0 and i == 0:
                #print(test1, cv_red, cv_norm, cv_mm)
        #print(t)
    et = time.time()
    # print(et-st)     
    ##########

    #st2 =time.time()            
    N=5000
    n_s = n_1 + n_2
    ####red       
    size_red  = torch.mean(res1_red, dim=0) 
    
    index_red = int(max(torch.where(abs(size_red-alpha) == min(abs(size_red-alpha)))[0]))
    #index_red = torch.argmin(abs(size_red - alpha)).item()
    n_tilde_red = int(n_tilde[index_red])
    M = fun_all.generate_one_minus_one_distribution(N, n_tilde_red).float().to(device)
    eps_t1 = eps_all[n_s:(n_s+n_tilde_red), :]
    delta_t1 = delta_all[n_s:(n_s+n_tilde_red), :]
    test_1 = torch.max(abs(eps_t1.T @ delta_t1 / np.sqrt(n_tilde_red) ))
    #############
    
    
    size_norm  = torch.mean(res1_norm, dim=0) 
    index_norm = int(max(torch.where(abs(size_norm-alpha) == min(abs(size_norm-alpha)))[0]))
    #index_norm = torch.argmin(abs(size_norm - alpha)).item()
    n_tilde_norm = int(n_tilde[index_norm])
    eps_t2 = eps_all[n_s:(n_s+n_tilde_norm), :]
    delta_t2 = delta_all[n_s:(n_s+n_tilde_norm), :]
    test_2 =  torch.max(abs(eps_t2.T @ delta_t2 / np.sqrt(n_tilde_norm) ))
    M_norm = torch.randn(N, n_tilde_norm).to(device)
    #norm 

    ####mannen       
    size_mm  = torch.mean(res1_mm, dim=0) 
    index_mm = int(max(torch.where(abs(size_mm-alpha) == min(abs(size_mm-alpha)))[0]))
    #index_mm = torch.argmin(abs(size_mm - alpha)).item()
    n_tilde_mm = int(n_tilde[index_mm])
    eps_t3 = eps_all[n_s:(n_s+n_tilde_mm), :]
    delta_t3 = delta_all[n_s:(n_s+n_tilde_mm), :]
    test_3 = torch.max(abs(eps_t3.T @ delta_t3 / np.sqrt(n_tilde_mm) ))
    M_mm = fun_all.generate_mammen(N,n_tilde_mm).float().to(device)
    
    
    n_tilde_red = torch.sqrt(torch.as_tensor(n_tilde_red)).to(device)
    n_tilde_norm = torch.sqrt(torch.as_tensor(n_tilde_norm)).to(device)
    n_tilde_mm = torch.sqrt(torch.as_tensor(n_tilde_mm)).to(device)
    dat_new1 = torch.empty(N, p).to(device)
    dat_new2 = torch.empty(N, p).to(device)
    dat_new3 = torch.empty(N, p).to(device)
    for i in range(p):
        dat_1 = eps_t1[:, i:i+1] * delta_t1  # Hadamard 内积，形状 (n_tilde1, q)
        dat_1 = dat_1 - torch.mean(dat_1, dim=0, keepdim=True)  # 减去均值
        dat_new1[:, i] = torch.max(abs(M @ dat_1 / n_tilde_red), axis=1)[0]  # 形状 (N,)

        dat_2 = eps_t2[:, i:i+1] * delta_t2  # Hadamard 内积，形状 (n_tilde1, q)
        dat_2 = dat_2 - torch.mean(dat_2, dim=0, keepdim=True)  # 减去均值
        dat_new2[:, i] = torch.max(abs(M_norm @ dat_2 / n_tilde_norm), axis=1)[0]  # 形状 (N,)
        
        dat_3 = eps_t3[:, i:i+1] * delta_t3  # Hadamard 内积，形状 (n_tilde1, q)
        dat_3 = dat_3 - torch.mean(dat_3, dim=0, keepdim=True)  # 减去均值
        dat_new3[:, i] = torch.max(abs(M_mm @ dat_3 / n_tilde_mm), axis=1)[0]  # 形状 (N,)
    # dat_1 = eps_t1.repeat_interleave(q, dim=1) * delta_t1.repeat(1, p)
    # dat_1 = dat_1 - torch.mean(dat_1, dim=0)
    #rademacher
    
    dat_new1 = torch.max(dat_new1, axis = 1).values
    dat_new2 = torch.max(dat_new2, axis = 1).values
    dat_new3 = torch.max(dat_new3, axis = 1).values
    # dat_new1 = torch.max(abs(M @ dat_1 / np.sqrt(n_tilde_red)), axis=1)[0]
    cv_red = torch.kthvalue(dat_new1, (N-int(N*alpha)+1)).values
    sA[0,0] = (test_1 > cv_red)
    sA[0,1] = size_red[index_red] 
    sA[0,2]= n_tilde[index_red]
    
    
    #st1 =time.time()
    
    #############
    # dat_2 = eps_t2.repeat_interleave(q, dim=1) * delta_t2.repeat(1, p)
    # dat_2 = dat_2 - torch.mean(dat_2, dim=0)
    # dat_new2 = torch.max(abs(M_norm @ dat_2 / np.sqrt(n_tilde_norm)), axis=1)[0]
    cv_norm = torch.kthvalue(dat_new2, (N-int(N*alpha)+1)).values
    #et1 =time.time()
    #print(et1-st1)
    sA[1,0] = (test_2 > cv_norm)
    sA[1,1] = size_norm[index_norm] 
    sA[1,2]= n_tilde[index_norm]
     
    
    
    #############
    # dat_3 = eps_t3.repeat_interleave(q, dim=1) * delta_t3.repeat(1, p)
    # dat_3 = dat_3 - torch.mean(dat_3, dim=0)
    # #norm 
    
    # dat_new3 = torch.max(abs(M_mm @ dat_3 / np.sqrt(n_tilde_mm)), axis=1)[0]
    cv_mm = torch.kthvalue(dat_new3, (N-int(N*alpha)+1)).values
    sA[2,0] = (test_3 > cv_mm)
    sA[2,1] = size_norm[index_mm] 
    sA[2,2]= n_tilde[index_mm]    
    
    #et2 =time.time()
    #print(et2-st2)
    
    return  sA 

In [6]:
args = Namespace(
    lr = 0.01,
    device = 'cuda'
)

In [7]:
p=400 
q=400 
m=5 
n = 200 
n_1 = int(n/3)
n_2 = int(n/2)
n_3 = n - n_1 -n_2

######n50
n_tilde = torch.arange(int(n_3/3), n_3, 2)
alpha=0.05


n_my = 500

In [8]:
# 实例化网络
input_features = m
hidden_features1 = int(128)
hidden_features2 = int(32)
output_features = 1
n_epochs = 400
patience = 30
batchsize = 16
val_pct = 0.15
lr = 0.01

In [9]:
def run_simulation(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, gene_data, para_time):
    logger = init_logging("logex6_n200_p400_12.log")  # 为每个任务初始化日志
    torch.manual_seed((i + 1) * 12345)  # 设置随机种子
    data_my = gene_data(n, p, q, m, dep)  # 生成数据
    
    tasks_per_machine = para_time // 8
    tmp = tasks_per_machine * 8 
    adjusted_index = (i - tmp if i > (tmp-1) else i) // tasks_per_machine
    machine_index = adjusted_index % 8
    device = args.device+':'+str(machine_index)
    
    x = data_my["x"] 
    y = data_my["y"] 
    z = data_my["z"] 
    # z = torch.randn(n,p)
    # x = torch.exp(- z**2/2)* torch.sin(2*z) +  torch.randn(n,p)  
    # y = torch.exp(- z**2/2)* torch.sin(2*z) +  torch.randn(n,p)  #+x
    logger.info(f"任务 {i} 完成")  # 写入日志
    result = gauss_fun(i,x, y, z, n, n_1, n_2, n_3, n_tilde, p, q, m, dep, alpha,batchsize,val_pct,lr,n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device)
    return result

In [ ]:
cuda_cores = 100  # GPU 核心数量
# cuda_cores = 100  # 可根据实际的 GPU 核心数量或 CPU 线程设置

# sim_list = {'my_data_ex10':0}
sim_list = {'my_data_ex6': 0,'my_data_ex7': 0,'my_data_ex8': 0,'my_data_ex9': 0,'my_data_ex10': 0}
for sim, dep in sim_list.items():
    resmy = torch.zeros(n_my, 3, 3)
    print(sim, dep)
    try:
        with Parallel(n_jobs=cuda_cores) as parallel:
            tmp_results = parallel(
                delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data_Copy1, sim), cuda_cores)
                for i in tqdm(range(n_my))
            )
        results = torch.stack(tmp_results).to('cpu')
        del tmp_results
        

        # results = torch.stack(Parallel(n_jobs=cuda_cores)(delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data, sim), cuda_cores) for i in tqdm(range(n_my))))
        res_all = torch.randn(n_my, 3)
        res_size = torch.randn(n_my, 3)
        res_n = torch.randn(n_my, 3)
        for i in range(n_my):
            res_all[i, :] = results[i, :, 0]
            res_size[i, :] = results[i, :, 1]
            res_n[i, :] = results[i, :, 2]
            
        res_my = torch.mean(res_all, dim=0)
        
        df1 = pd.DataFrame(res_my)
    
        dic_path = './ind1_test_m52'
        file_path = os.path.join(dic_path, f'result_{sim}1.xlsx')
        
        # 将DataFrame保存到Excel文件
        # df1.to_excel(file_path, index=False, sheet_name='Sheet1')

        # 将 DataFrame 保存到 CSV 文件
        file_path = os.path.join(dic_path, f'result_{sim}.csv')
        df1.to_csv(file_path, index=False, encoding='utf-8')
        
        
        df2 = pd.DataFrame(res_size)
        # file_path2 = os.path.join(dic_path, f'result_{sim}power.xlsx')
        file_path2 = os.path.join(dic_path, f'result_{sim}size.csv')
        # 将DataFrame保存到Excel文件
        # df2.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df2.to_csv(file_path2, index=False, encoding='utf-8')
        
        df3 = pd.DataFrame(res_n)
        # file_path3 = os.path.join(dic_path, f'result_{sim}_n1.xlsx')
        file_path3 = os.path.join(dic_path, f'result_{sim}_n.csv')
        
        # 将DataFrame保存到Excel文件
        # df3.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df3.to_csv(file_path3, index=False, encoding='utf-8')
        print(res_my)
        send_email(f"48号机{sim}的size程序运行成功", str(res_my))

        del results
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    except Exception as e:
        # 捕获异常并发送邮件
        error_message = f"程序运行出错:\n{traceback.format_exc()}"
        print(error_message)
        send_email(f"48号机{sim}的size程序出错通知", error_message)

my_data_ex6 0


  0%|          | 0/500 [00:00<?, ?it/s]

2025-01-23 03:38:21,700 - 任务 8 完成
2025-01-23 03:38:21,761 - 任务 14 完成
2025-01-23 03:38:21,765 - 任务 13 完成
2025-01-23 03:38:21,783 - 任务 10 完成
2025-01-23 03:38:21,785 - 任务 22 完成
2025-01-23 03:38:21,794 - 任务 2 完成
2025-01-23 03:38:21,797 - 任务 16 完成
2025-01-23 03:38:21,821 - 任务 18 完成
2025-01-23 03:38:21,832 - 任务 21 完成
2025-01-23 03:38:21,894 - 任务 24 完成
2025-01-23 03:38:22,014 - 任务 55 完成
2025-01-23 03:38:22,067 - 任务 56 完成
2025-01-23 03:38:22,087 - 任务 47 完成
2025-01-23 03:38:22,101 - 任务 61 完成
2025-01-23 03:38:22,136 - 任务 71 完成
2025-01-23 03:38:22,140 - 任务 4 完成
2025-01-23 03:38:22,209 - 任务 1 完成
2025-01-23 03:38:22,244 - 任务 76 完成
2025-01-23 03:38:22,251 - 任务 6 完成
2025-01-23 03:38:22,267 - 任务 3 完成
2025-01-23 03:38:22,270 - 任务 0 完成
2025-01-23 03:38:22,274 - 任务 9 完成
2025-01-23 03:38:22,286 - 任务 69 完成
2025-01-23 03:38:22,307 - 任务 70 完成
2025-01-23 03:38:22,310 - 任务 75 完成
2025-01-23 03:38:22,314 - 任务 74 完成
2025-01-23 03:38:22,333 - 任务 72 完成
2025-01-23 03:38:22,333 - 任务 12 完成
2025-01-23 03:38:22,344 - 任务

In [ ]:
cuda_cores = 100  # GPU 核心数量
# cuda_cores = 100  # 可根据实际的 GPU 核心数量或 CPU 线程设置

# sim_list = {'my_data_ex7':int(5)}
sim_list = {'my_data_ex6': int(p/10),'my_data_ex7': int(7),'my_data_ex8': int(p/10),'my_data_ex9': int(p/10),'my_data_ex10': int(p/10)}
for sim, dep in sim_list.items():
    resmy = torch.zeros(n_my, 3, 3)
    print(sim, dep)
    try:
        with Parallel(n_jobs=cuda_cores) as parallel:
            tmp_results = parallel(       
                delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data_Copy1, sim), cuda_cores)
                for i in tqdm(range(n_my))
            )
        results = torch.stack(tmp_results).to('cpu')
        del tmp_results
        

        # results = torch.stack(Parallel(n_jobs=cuda_cores)(delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data, sim), cuda_cores) for i in tqdm(range(n_my))))
        res_all = torch.randn(n_my, 3)
        res_size = torch.randn(n_my, 3)
        res_n = torch.randn(n_my, 3)
        for i in range(n_my):
            res_all[i, :] = results[i, :, 0]
            res_size[i, :] = results[i, :, 1]
            res_n[i, :] = results[i, :, 2]
            
        res_my = torch.mean(res_all, dim=0)
        
        df1 = pd.DataFrame(res_my)
    
        dic_path = './ind1_test_m52'
        file_path = os.path.join(dic_path, f'result_{sim}1.xlsx')
        
        # 将DataFrame保存到Excel文件
        # df1.to_excel(file_path, index=False, sheet_name='Sheet1')

        # 将 DataFrame 保存到 CSV 文件
        file_path = os.path.join(dic_path, f'result_{sim}1.csv')
        df1.to_csv(file_path, index=False, encoding='utf-8')
        
        
        df2 = pd.DataFrame(res_size)
        # file_path2 = os.path.join(dic_path, f'result_{sim}power.xlsx')
        file_path2 = os.path.join(dic_path, f'result_{sim}power.csv')
        # 将DataFrame保存到Excel文件
        # df2.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df2.to_csv(file_path2, index=False, encoding='utf-8')
        
        df3 = pd.DataFrame(res_n)
        # file_path3 = os.path.join(dic_path, f'result_{sim}_n1.xlsx')
        file_path3 = os.path.join(dic_path, f'result_{sim}_n1.csv')
        
        # 将DataFrame保存到Excel文件
        # df3.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df3.to_csv(file_path3, index=False, encoding='utf-8')
        print(res_my)
        send_email(f"48号机{sim}的power程序运行成功", str(res_my))

        del results
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    except Exception as e:
        # 捕获异常并发送邮件
        error_message = f"程序运行出错:\n{traceback.format_exc()}"
        print(error_message)
        send_email(f"48号机{sim}的power程序出错通知", error_message)

In [ ]:
cuda_cores = 100  # GPU 核心数量
# cuda_cores = 100  # 可根据实际的 GPU 核心数量或 CPU 线程设置

# sim_list = {'my_data_ex7':int(5)}
sim_list = {'my_data_ex6': int(p/5),'my_data_ex7': int(8),'my_data_ex8': int(p/5),'my_data_ex9': int(p/5),'my_data_ex10': int(p/5)}
for sim, dep in sim_list.items():
    resmy = torch.zeros(n_my, 3, 3)
    print(sim, dep)
    try:
        with Parallel(n_jobs=cuda_cores) as parallel:
            tmp_results = parallel(       
                delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data_Copy1, sim), cuda_cores)
                for i in tqdm(range(n_my))
            )
        results = torch.stack(tmp_results).to('cpu')
        del tmp_results
        

        # results = torch.stack(Parallel(n_jobs=cuda_cores)(delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data, sim), cuda_cores) for i in tqdm(range(n_my))))
        res_all = torch.randn(n_my, 3)
        res_size = torch.randn(n_my, 3)
        res_n = torch.randn(n_my, 3)
        for i in range(n_my):
            res_all[i, :] = results[i, :, 0]
            res_size[i, :] = results[i, :, 1]
            res_n[i, :] = results[i, :, 2]
            
        res_my = torch.mean(res_all, dim=0)
        
        df1 = pd.DataFrame(res_my)
    
        dic_path = './ind1_test_m52'
        file_path = os.path.join(dic_path, f'result_{sim}2.xlsx')
        
        # 将DataFrame保存到Excel文件
        # df1.to_excel(file_path, index=False, sheet_name='Sheet1')

        # 将 DataFrame 保存到 CSV 文件
        file_path = os.path.join(dic_path, f'result_{sim}2.csv')
        df1.to_csv(file_path, index=False, encoding='utf-8')
        
        
        df2 = pd.DataFrame(res_size)
        # file_path2 = os.path.join(dic_path, f'result_{sim}power.xlsx')
        file_path2 = os.path.join(dic_path, f'result_{sim}power2.csv')
        # 将DataFrame保存到Excel文件
        # df2.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df2.to_csv(file_path2, index=False, encoding='utf-8')
        
        df3 = pd.DataFrame(res_n)
        # file_path3 = os.path.join(dic_path, f'result_{sim}_n1.xlsx')
        file_path3 = os.path.join(dic_path, f'result_{sim}_n2.csv')
        
        # 将DataFrame保存到Excel文件
        # df3.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df3.to_csv(file_path3, index=False, encoding='utf-8')
        print(res_my)
        send_email(f"48号机{sim}的power程序运行成功", str(res_my))

        del results
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    except Exception as e:
        # 捕获异常并发送邮件
        error_message = f"程序运行出错:\n{traceback.format_exc()}"
        print(error_message)
        send_email(f"48号机{sim}的power程序出错通知", error_message)